In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from itertools import chain
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
import numpy as np

/home/saurabhjhanjee/anaconda3/envs/applied-data-science1/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
FLIGHT_DATA_PATH = "../data/raw/APM_Report_Formatted_2021.csv"
AIRPORT_CODE_MAP = {132: "JFK", 1: "EWR", 138: "LGA"}

flight_df = pd.read_csv(FLIGHT_DATA_PATH)
flight_df.head()

,Facility,Date,Hour,GMT Hour,Scheduled Departures,Scheduled Arrivals,Departures For Metric Computation,Arrivals For Metric Computation,% On-Time Gate Departures,% On-Time Airport Departures,% On-Time Gate Arrivals,Average Gate Departure Delay,Average Taxi Out Time,Average Taxi Out Delay,Average Airport Departure Delay,Average Airborne Delay,Average Taxi In Delay,Average Block Delay,Average Gate Arrival Delay
0,EWR,01/01/2021,0,5,1,3,1,1,100.0,100.0,100.0,-3.0,10.0,-2.6,-5.6,-8.0,1.50,-14.00,-29.00
1,EWR,01/01/2021,1,6,0,1,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.75,13.00,26.85
2,EWR,01/01/2021,2,7,0,0,0,1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,1.0,0.30,3.00,-61.30
3,EWR,01/01/2021,4,9,0,2,0,2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,-23.0,-1.70,-16.00,-37.00
4,EWR,01/01/2021,5,10,1,6,1,4,100.0,100.0,100.0,0.0,12.0,-2.5,-2.5,-10.0,0.55,-1.75,-11.50


In [5]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Ass1 ESA")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/21 02:01:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sdf = spark.read.parquet('../data/raw/fhvhv')

In [5]:
# sdf.sort(F.col("request_datetime")).head(10)

In [7]:
flight_code_mapper = F.create_map([F.lit(x) for x in chain(*AIRPORT_CODE_MAP.items())])

In [8]:
sdf = sdf.filter(F.col("base_passenger_fare") > 0)

In [9]:
from_airport = sdf.filter(F.col("PULocationID").isin(list(AIRPORT_CODE_MAP.keys())))
from_airport = from_airport.withColumn("Airport", flight_code_mapper[F.col("PULocationID")])


In [11]:
to_airport = sdf.filter(F.col("DOLocationID").isin(list(AIRPORT_CODE_MAP.keys())))
to_airport = to_airport.withColumn("Airport", flight_code_mapper[F.col("DOLocationID")])
to_airport.limit(10)

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,Airport
HV0003,B02872,B02872,2021-07-01 00:06:43,2021-07-01 00:24:14,2021-07-01 00:26:35,2021-07-01 00:33:38,132,132,2.68,424,9.49,0.0,0.36,1.06,0.0,2.5,0.0,16.38,N,N,,N,N,JFK
HV0003,B02395,B02395,2021-07-01 01:00:00,2021-07-01 00:55:50,2021-07-01 00:55:57,2021-07-01 01:30:02,243,132,21.04,2045,68.95,6.55,0.0,0.0,0.0,2.5,41.31,50.75,N,N,,N,N,JFK
HV0005,B02510,null,2021-07-01 00:45:29,null,2021-07-01 00:54:45,2021-07-01 00:57:42,132,132,0.559,177,12.41,0.0,0.45,1.32,0.0,2.5,0.0,5.47,N,N,N,N,N,JFK
HV0005,B02510,null,2021-07-01 00:38:57,null,2021-07-01 00:42:52,2021-07-01 00:48:15,132,132,0.449,323,18.25,0.0,0.62,1.84,0.0,2.5,0.0,17.71,N,N,N,N,N,JFK
HV0003,B02836,B02836,2021-07-01 00:32:44,2021-07-01 00:49:04,2021-07-01 00:49:08,2021-07-01 01:07:54,130,132,6.83,1126,27.97,0.0,0.91,2.7,0.0,2.5,0.0,19.1,N,N,,N,N,JFK
HV0003,B02882,B02882,2021-07-01 00:03:48,2021-07-01 00:08:34,2021-07-01 00:10:11,2021-07-01 00:50:31,188,132,11.87,2420,45.24,0.0,1.43,4.24,0.0,2.5,0.0,34.92,N,N,,N,N,JFK
HV0003,B02617,B02617,2021-06-30 23:57:09,2021-06-30 23:59:31,2021-07-01 00:01:31,2021-07-01 00:09:27,138,138,0.88,476,17.72,0.0,0.61,1.79,0.0,2.5,0.0,17.28,N,N,,N,N,LGA
HV0003,B02836,B02836,2021-07-01 00:40:34,2021-07-01 00:42:15,2021-07-01 00:43:05,2021-07-01 00:52:15,92,138,3.05,550,9.69,0.0,0.37,1.08,0.0,2.5,0.0,9.23,N,N,,N,N,LGA
HV0003,B02879,B02879,2021-07-01 00:04:59,2021-07-01 00:14:17,2021-07-01 00:16:17,2021-07-01 00:25:55,223,138,3.16,578,14.55,0.0,0.51,1.51,0.0,2.5,1.5,10.21,N,N,,N,N,LGA
HV0005,B02510,null,2021-06-30 23:58:34,null,2021-07-01 00:02:54,2021-07-01 00:28:00,25,138,11.336,1506,32.88,0.0,0.91,2.7,0.0,2.5,0.0,25.2,N,N,N,N,N,LGA


In [12]:
flight_df.dtypes

Facility                              object
Date                                  object
Hour                                   int64
GMT Hour                               int64
Scheduled Departures                   int64
Scheduled Arrivals                     int64
Departures For Metric Computation      int64
Arrivals For Metric Computation        int64
% On-Time Gate Departures            float64
% On-Time Airport Departures         float64
% On-Time Gate Arrivals              float64
Average Gate Departure Delay         float64
Average Taxi Out Time                float64
Average Taxi Out Delay               float64
Average Airport Departure Delay      float64
Average Airborne Delay               float64
Average Taxi In Delay                float64
Average Block Delay                  float64
Average Gate Arrival Delay           float64
dtype: object

In [20]:
def preprocess_temporal_columns(sdf):
    hour_intervals = range(1,6)
    cols = []
    for interval in hour_intervals:
        cols += [[f"+{interval} Hour", f"+{interval} Date"], [f"-{interval} Hour", f"-{interval} Date"]]
        sdf = sdf.withColumn(f"+{interval} Hour", F.hour(F.col("request_datetime") + F.expr(f'INTERVAL {interval} HOURS')))
        sdf = sdf.withColumn(f"+{interval} Date", F.to_date(F.col("request_datetime") + F.expr(f'INTERVAL {interval} HOURS')))

        sdf = sdf.withColumn(f"-{interval} Hour", F.hour(F.col("request_datetime") + F.expr(f'INTERVAL -{interval} HOURS')))
        sdf = sdf.withColumn(f"-{interval} Date", F.to_date(F.col("request_datetime")+ F.expr(f'INTERVAL -{interval} HOURS')))


    sdf = sdf.withColumn("Day", F.dayofweek(F.col("request_datetime")))
    sdf = sdf.withColumn("Hour", F.hour(F.col("request_datetime")))
    sdf = sdf.withColumn("Date", F.to_date(F.col("request_datetime")))
    return sdf, cols

def preprocess_columns(sdf):
    sdf, cols = preprocess_temporal_columns(sdf)
    cols_flat =  [item for sublist in cols for item in sublist]
    return sdf.groupby(*("Day", "Hour", "Date", "Airport",  *cols_flat)).count(), cols


In [21]:
from_grouped, cols = preprocess_columns(from_airport)
cols_flat =  [item for sublist in cols for item in sublist]
from_grouped.limit(10)

Day,Hour,Date,Airport,+1 Hour,+1 Date,-1 Hour,-1 Date,+2 Hour,+2 Date,-2 Hour,-2 Date,+3 Hour,+3 Date,-3 Hour,-3 Date,+4 Hour,+4 Date,-4 Hour,-4 Date,+5 Hour,+5 Date,-5 Hour,-5 Date,count
7,6,2021-07-03,LGA,7,2021-07-03,5,2021-07-03,8,2021-07-03,4,2021-07-03,9,2021-07-03,3,2021-07-03,10,2021-07-03,2,2021-07-03,11,2021-07-03,1,2021-07-03,5
7,18,2021-07-10,LGA,19,2021-07-10,17,2021-07-10,20,2021-07-10,16,2021-07-10,21,2021-07-10,15,2021-07-10,22,2021-07-10,14,2021-07-10,23,2021-07-10,13,2021-07-10,347
3,6,2021-07-13,LGA,7,2021-07-13,5,2021-07-13,8,2021-07-13,4,2021-07-13,9,2021-07-13,3,2021-07-13,10,2021-07-13,2,2021-07-13,11,2021-07-13,1,2021-07-13,12
4,5,2021-07-14,JFK,6,2021-07-14,4,2021-07-14,7,2021-07-14,3,2021-07-14,8,2021-07-14,2,2021-07-14,9,2021-07-14,1,2021-07-14,10,2021-07-14,0,2021-07-14,136
5,7,2021-07-15,JFK,8,2021-07-15,6,2021-07-15,9,2021-07-15,5,2021-07-15,10,2021-07-15,4,2021-07-15,11,2021-07-15,3,2021-07-15,12,2021-07-15,2,2021-07-15,259
1,0,2021-07-18,LGA,1,2021-07-18,23,2021-07-17,2,2021-07-18,22,2021-07-17,3,2021-07-18,21,2021-07-17,4,2021-07-18,20,2021-07-17,5,2021-07-18,19,2021-07-17,99
1,12,2021-07-25,JFK,13,2021-07-25,11,2021-07-25,14,2021-07-25,10,2021-07-25,15,2021-07-25,9,2021-07-25,16,2021-07-25,8,2021-07-25,17,2021-07-25,7,2021-07-25,320
1,21,2021-07-25,JFK,22,2021-07-25,20,2021-07-25,23,2021-07-25,19,2021-07-25,0,2021-07-26,18,2021-07-25,1,2021-07-26,17,2021-07-25,2,2021-07-26,16,2021-07-25,604
7,13,2021-07-10,LGA,14,2021-07-10,12,2021-07-10,15,2021-07-10,11,2021-07-10,16,2021-07-10,10,2021-07-10,17,2021-07-10,9,2021-07-10,18,2021-07-10,8,2021-07-10,255
1,19,2021-07-11,JFK,20,2021-07-11,18,2021-07-11,21,2021-07-11,17,2021-07-11,22,2021-07-11,16,2021-07-11,23,2021-07-11,15,2021-07-11,0,2021-07-12,14,2021-07-11,528


In [22]:
to_grouped, cols = preprocess_columns(to_airport)
to_grouped.limit(10)

Day,Hour,Date,Airport,+1 Hour,+1 Date,-1 Hour,-1 Date,+2 Hour,+2 Date,-2 Hour,-2 Date,+3 Hour,+3 Date,-3 Hour,-3 Date,+4 Hour,+4 Date,-4 Hour,-4 Date,+5 Hour,+5 Date,-5 Hour,-5 Date,count
7,6,2021-07-03,LGA,7,2021-07-03,5,2021-07-03,8,2021-07-03,4,2021-07-03,9,2021-07-03,3,2021-07-03,10,2021-07-03,2,2021-07-03,11,2021-07-03,1,2021-07-03,443
2,11,2021-07-05,EWR,12,2021-07-05,10,2021-07-05,13,2021-07-05,9,2021-07-05,14,2021-07-05,8,2021-07-05,15,2021-07-05,7,2021-07-05,16,2021-07-05,6,2021-07-05,125
7,18,2021-07-10,LGA,19,2021-07-10,17,2021-07-10,20,2021-07-10,16,2021-07-10,21,2021-07-10,15,2021-07-10,22,2021-07-10,14,2021-07-10,23,2021-07-10,13,2021-07-10,77
1,10,2021-07-11,EWR,11,2021-07-11,9,2021-07-11,12,2021-07-11,8,2021-07-11,13,2021-07-11,7,2021-07-11,14,2021-07-11,6,2021-07-11,15,2021-07-11,5,2021-07-11,98
3,6,2021-07-13,LGA,7,2021-07-13,5,2021-07-13,8,2021-07-13,4,2021-07-13,9,2021-07-13,3,2021-07-13,10,2021-07-13,2,2021-07-13,11,2021-07-13,1,2021-07-13,540
4,5,2021-07-14,JFK,6,2021-07-14,4,2021-07-14,7,2021-07-14,3,2021-07-14,8,2021-07-14,2,2021-07-14,9,2021-07-14,1,2021-07-14,10,2021-07-14,0,2021-07-14,896
4,12,2021-07-14,EWR,13,2021-07-14,11,2021-07-14,14,2021-07-14,10,2021-07-14,15,2021-07-14,9,2021-07-14,16,2021-07-14,8,2021-07-14,17,2021-07-14,7,2021-07-14,139
5,7,2021-07-15,JFK,8,2021-07-15,6,2021-07-15,9,2021-07-15,5,2021-07-15,10,2021-07-15,4,2021-07-15,11,2021-07-15,3,2021-07-15,12,2021-07-15,2,2021-07-15,439
7,1,2021-07-17,EWR,2,2021-07-17,0,2021-07-17,3,2021-07-17,23,2021-07-16,4,2021-07-17,22,2021-07-16,5,2021-07-17,21,2021-07-16,6,2021-07-17,20,2021-07-16,2
1,0,2021-07-18,LGA,1,2021-07-18,23,2021-07-17,2,2021-07-18,22,2021-07-17,3,2021-07-18,21,2021-07-17,4,2021-07-18,20,2021-07-17,5,2021-07-18,19,2021-07-17,8


In [24]:
from_df = from_grouped.toPandas().dropna()
to_df = to_grouped.toPandas().dropna()

In [25]:
from_df = from_df.astype({"Hour": "int32", "Date": 'datetime64[ns]', **{item: "int32" if item.endswith("Hour") else 'datetime64[ns]' for item in cols_flat}})
to_df = to_df.astype({"Hour": "int32", "Date": 'datetime64[ns]', **{item: "int32" if item.endswith("Hour") else 'datetime64[ns]' for item in cols_flat}})

flight_df["Date"] = pd.to_datetime(flight_df["Date"])

In [40]:
feature_list = ["Day", "Departures For Metric Computation", "Arrivals For Metric Computation", "Hour", "Facility", "count", "Date"]
for col_pair in cols:
    for item in col_pair:
        feature_list.append(f"Departures For Metric Computation{item[:2]}")
        feature_list.append(f"Arrivals For Metric Computation{item[:2]}")
epsilon = 0.1

def join_data(taxis, flights):
    joined_df = flights.merge(taxis, left_on=("Hour", "Date", "Facility"), right_on=("Hour", "Date", "Airport"), how="inner")
    for col_pair in cols:
        joined_df = joined_df.merge(flights, left_on=(*col_pair, "Airport"), right_on=("Hour", "Date", "Facility"), how="inner", suffixes=(None, col_pair[0][:2]))
        # joined_df[f"Departures For Metric Computation{col_pair[0][:2]}"] = np.power(np.e, joined_df[f"Departures For Metric Computation{col_pair[0][:2]}"])
        # joined_df[f"Arrivals For Metric Computation{col_pair[0][:2]}"] = joined_df[f"Arrivals For Metric Computation{col_pair[0][:2]}"]
    joined_df = joined_df[feature_list]
    joined_df = joined_df.loc[:,~joined_df.columns.duplicated()]        
    return joined_df
cols

[['+1 Hour', '+1 Date'],
 ['-1 Hour', '-1 Date'],
 ['+2 Hour', '+2 Date'],
 ['-2 Hour', '-2 Date'],
 ['+3 Hour', '+3 Date'],
 ['-3 Hour', '-3 Date'],
 ['+4 Hour', '+4 Date'],
 ['-4 Hour', '-4 Date'],
 ['+5 Hour', '+5 Date'],
 ['-5 Hour', '-5 Date']]

In [41]:
data = join_data(to_df, flight_df)
data

,Day,Departures For Metric Computation,Arrivals For Metric Computation,Hour,Facility,count,Date,Departures For Metric Computation+1,Arrivals For Metric Computation+1,Departures For Metric Computation-1,...,Departures For Metric Computation-3,Arrivals For Metric Computation-3,Departures For Metric Computation+4,Arrivals For Metric Computation+4,Departures For Metric Computation-4,Arrivals For Metric Computation-4,Departures For Metric Computation+5,Arrivals For Metric Computation+5,Departures For Metric Computation-5,Arrivals For Metric Computation-5
0,6,12,4,9,EWR,48,2021-01-01,22,10,20,...,8,4,5,19,1,4,12,20,0,2
1,6,22,10,10,EWR,54,2021-01-01,12,7,12,...,14,5,12,20,8,4,23,12,1,4
2,6,12,7,11,EWR,51,2021-01-01,11,8,22,...,20,5,23,12,14,5,13,19,8,4
3,6,11,8,12,EWR,48,2021-01-01,5,19,12,...,12,4,13,19,20,5,8,27,14,5
4,6,5,19,13,EWR,47,2021-01-01,12,20,11,...,22,10,8,27,12,4,28,15,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11121,7,14,19,14,LGA,360,2021-07-31,20,19,13,...,13,15,10,17,18,13,14,13,21,16
11122,7,20,19,15,LGA,310,2021-07-31,17,12,14,...,13,9,14,13,13,15,6,14,18,13
11123,7,17,12,16,LGA,190,2021-07-31,14,11,20,...,13,18,6,14,13,9,2,8,13,15
11124,7,14,11,17,LGA,140,2021-07-31,10,17,17,...,14,19,2,8,13,18,1,10,13,9


In [42]:
y = data["count"]

X = pd.get_dummies(data, columns=["Day", "Facility"]).drop(["count"], axis=1)

pipeline = make_pipeline(PolynomialFeatures(interaction_only=True, include_bias = False), StandardScaler(), ElasticNet())

X_train,X_test,y_train,y_test=train_test_split(X,y)


In [43]:
params =  {"elasticnet__alpha": [.5, 1, 10],
                      "elasticnet__l1_ratio": [1]}

gs = GridSearchCV(pipeline, params, scoring="r2")

gs.fit(X_train.drop(["Date", "Hour"], axis=1), y_train)

GridSearchCV(estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(include_bias=False,
                                                           interaction_only=True)),
                                       ('standardscaler', StandardScaler()),
                                       ('elasticnet', ElasticNet())]),
             param_grid={'elasticnet__alpha': [0.5, 1, 10],
                         'elasticnet__l1_ratio': [1]},
             scoring='r2')

In [47]:
X_train.drop(["Date", "Hour"], axis=1).columns

Index(['Departures For Metric Computation', 'Arrivals For Metric Computation',
       'Departures For Metric Computation+1',
       'Arrivals For Metric Computation+1',
       'Departures For Metric Computation-1',
       'Arrivals For Metric Computation-1',
       'Departures For Metric Computation+2',
       'Arrivals For Metric Computation+2',
       'Departures For Metric Computation-2',
       'Arrivals For Metric Computation-2',
       'Departures For Metric Computation+3',
       'Arrivals For Metric Computation+3',
       'Departures For Metric Computation-3',
       'Arrivals For Metric Computation-3',
       'Departures For Metric Computation+4',
       'Arrivals For Metric Computation+4',
       'Departures For Metric Computation-4',
       'Arrivals For Metric Computation-4',
       'Departures For Metric Computation+5',
       'Arrivals For Metric Computation+5',
       'Departures For Metric Computation-5',
       'Arrivals For Metric Computation-5', 'Day_1', 'Day_2', 'Da

In [48]:
gs.best_params_

{'elasticnet__alpha': 0.5, 'elasticnet__l1_ratio': 1}

In [49]:
r2_score(y_test, gs.best_estimator_.predict(X_test.drop(["Date", "Hour"], axis=1)))


0.9486781089467528

In [50]:
mean_squared_error(y_test, gs.best_estimator_.predict(X_test.drop(["Date", "Hour"], axis=1)))


1346.3437072444024

In [51]:
X_test = pd.read_parquet("../data/curated/test/Airport Dropoffs_2022_test")
y_test = X_test["count"]
X_test = pd.get_dummies(X_test, columns=["Day", "Facility"]).drop("count", axis=1)
X_test

,Departures For Metric Computation,Arrivals For Metric Computation,Hour,Date,Departures For Metric Computation+1,Arrivals For Metric Computation+1,Departures For Metric Computation-1,Arrivals For Metric Computation-1,Departures For Metric Computation+2,Arrivals For Metric Computation+2,...,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Facility_EWR,Facility_JFK,Facility_LGA
0,18,25,12,2022-02-01,19,32,26,23,33,34,...,0,0,1,0,0,0,0,1,0,0
1,27,43,19,2022-02-01,32,33,44,23,28,27,...,0,0,1,0,0,0,0,1,0,0
2,24,31,20,2022-02-17,23,32,30,26,2,16,...,0,0,0,0,1,0,0,0,0,1
3,4,13,0,2022-02-21,5,2,8,20,1,3,...,0,1,0,0,0,0,0,0,1,0
4,33,33,14,2022-02-22,33,34,32,38,32,30,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,2,1,4,2022-04-05,5,25,1,2,26,22,...,0,0,1,0,0,0,0,0,1,0
5045,32,34,16,2022-04-06,35,37,36,29,35,34,...,0,0,0,1,0,0,0,0,0,1
5046,26,33,16,2022-04-14,33,28,31,24,32,29,...,0,0,0,0,1,0,0,0,1,0
5047,33,37,11,2022-04-22,20,26,42,23,29,39,...,0,0,0,0,0,1,0,1,0,0


In [52]:
pipeline = make_pipeline(PolynomialFeatures(interaction_only=True, include_bias = False), StandardScaler(), ElasticNet(alpha=0.5, l1_ratio=1))
pipeline.fit(X_train.drop(["Date", "Hour"], axis=1), y_train)
pred = pipeline.predict(X_test.drop(["Date", "Hour"], axis=1))
r2_score(y_test, pred)


0.8055515343531785

In [39]:
test = X_test.copy()

In [26]:
test["datetime"] = pd.to_datetime(test["Date"].astype(str) + test["Hour"].astype(str), format='%Y-%m-%d%H')

In [27]:
pipeline.get_params()["elasticnet"].coef_

array([-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        4.92012249e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.57141687e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  3.63517347e+00, -0.00000000e+00,
       -0.00000000e+00,  1.05100048e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        1.48063532e+01,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -

KeyError: "['datetime'] not found in axis"

In [ ]:
import datetime

plt.rcParams["figure.figsize"] = (60,10)
df = pd.concat((test, y_test), axis=1)
colours = ["red", "blue", "green"]
for i, airport in enumerate(AIRPORT_CODE_MAP.values()):
    curr = df[df[f"Facility_{airport}"] == 1].sort_values("datetime")
    preds = gs.best_estimator_.predict(curr.drop(["Date", "Hour", "datetime", "count"], axis=1))
    plt.scatter(curr["datetime"], curr["count"], color=colours[i])
    plt.plot(curr["datetime"], preds, color=colours[i])

plt.xlim((datetime.date(2021, 1, 1), datetime.date(2021, 3, 30)))
plt.show()


In [ ]:
plt.matshow(df.corr())
cb = plt.colorbar()


In [ ]:
df.shape

In [ ]:
X_test

In [ ]:

sdf2 = sdf.withColumn("Day", F.dayofweek(F.col("request_datetime")))
sdf2 = sdf2.withColumn("Date", F.to_date(F.col("request_datetime")))
sdf2 = sdf2.withColumn("Hour", F.hour(F.col("request_datetime")))

data2 = sdf2.groupby("Day", "Date").count().toPandas()


In [ ]:
plt.boxplot([data2[data2["Hour"] == i]["count"] for i in range(0, 24)])
plt.show()

In [ ]:
plt.boxplot([data2[data2["Day"] == i]["count"] for i in range(1, 8)])
plt.show()

In [37]:
params = {"svr__epsilon": [10],
           'svr__C':[500]}

svr_pipeline = make_pipeline(StandardScaler(), SVR())

gs = GridSearchCV(svr_pipeline, params)

gs.fit(X_train.drop(["Date", "Hour"], axis=1), y_train)

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svr', SVR())]),
             param_grid={'svr__C': [500], 'svr__epsilon': [10]})

In [38]:
r2_score(y_test, gs.best_estimator_.predict(X_test.drop(["Date", "Hour"], axis=1)))

0.5871188155379887

In [40]:
gs.best_score_

0.9660756612817194

In [41]:
gs.best_params_

{'svr__C': 500, 'svr__epsilon': 10}

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
for col in X_train.columns:
    plt.title(col)
    plt.scatter(X_train[col], y_train)
    plt.show()